In [1]:
import os
import re
import scipy
from scipy import stats
import pickle
import subprocess
import shlex
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.width', 1000)

In [3]:
Cat0_train = pd.read_csv('train0_8-2split.csv')
Cat0_test = pd.read_csv('test0_8-2split.csv')
Cat1_train = pd.read_csv('train1_8-2split.csv')
Cat1_test = pd.read_csv('test1_8-2split.csv')


In [4]:
print(Cat0_train.shape)
Cat0_train.head()

(7136, 67)


,id,hosp_admittime,hosp_dischtime,icu_intime,icu_outtime,los_icu,icu_death,gender,admission_age,weight_admit,height,charlson_score,atrial_fibrillation,malignant_cancer,chf,ckd,cld,copd,diabetes,hypertension,ihd,stroke,icu_outcome,race_encode_African,race_encode_Asian,race_encode_Caucasian,race_encode_Hispanic,race_encode_Not Specified,race_encode_South American,admission_type_DIRECT EMER.,admission_type_DIRECT OBSERVATION,admission_type_ELECTIVE,admission_type_EU OBSERVATION,admission_type_EW EMER.,admission_type_OBSERVATION ADMIT,admission_type_SURGICAL SAME DAY ADMISSION,admission_type_URGENT,first_careunit_Cardiac Vascular Intensive Care Unit (CVICU),first_careunit_Coronary Care Unit (CCU),first_careunit_Medical Intensive Care Unit (MICU),first_careunit_Medical/Surgical Intensive Care Unit (MICU/SICU),first_careunit_Neuro Intermediate,first_careunit_Neuro Stepdown,first_careunit_Neuro Surgical Intensive Care Unit (Neuro SICU),first_careunit_Surgical Intensive Care Unit (SICU),first_careunit_Trauma SICU (TSICU),aniongap,bicarbonate,bun,calcium,chloride,creatinine,glucose,sodium,potassium,hematocrit,hemoglobin,mch,mchc,mcv,platelet,rbc,rdw,wbc,inr,pt,ptt
0,20007037,2029-11-14 14:42:00,2029-11-16 16:51:00,2029-11-14 14:43:00,2029-11-15 16:22:00,1.07,0,1,58.869595,70.2,170.0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,12.5,25.500000,14.500000,8.400000,103.000000,1.200000,133.000000,137.0,4.1000,30.650,10.400000,30.550,33.9500,89.500000,191.500000,3.43000,15.200,8.100000,1.400000,15.000000,33.500000
1,20008724,2024-02-26 13:13:00,2024-03-01 13:20:00,2024-02-26 14:50:00,2024-02-27 23:45:00,1.37,0,0,47.154831,60.5,170.0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,12.0,25.000000,14.000000,9.000000,103.000000,0.500000,112.000000,137.0,3.4000,36.800,12.700000,33.500,34.4000,97.000000,294.000000,3.79000,12.900,9.200000,1.000000,11.000000,30.400000
2,20009550,2020-11-03 12:09:00,2020-11-10 16:53:00,2020-11-06 09:43:00,2020-11-08 15:45:00,2.25,0,0,52.841925,81.4,170.0,6,0,0,1,1,0,0,1,0,0,0,2,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,21.0,27.333333,49.333333,9.466667,97.666667,5.566667,174.333333,142.0,4.0000,37.800,11.933333,29.800,31.7000,93.666667,220.666667,4.03000,16.100,9.433333,1.366667,14.966667,71.066667
3,20012928,2023-02-26 01:19:00,2023-03-11 11:50:00,2023-02-26 16:04:00,2023-02-28 15:33:00,1.98,0,1,67.152148,91.8,170.0,4,0,0,0,1,0,0,0,1,0,0,2,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,21.5,24.250000,51.500000,8.512500,95.375000,13.487500,117.875000,137.0,6.2375,28.275,9.087500,28.600,32.1125,88.375000,203.875000,3.19875,15.975,8.925000,1.350000,14.450000,33.800000
4,20013244,1980-01-03 23:12:00,1980-01-15 18:15:00,1980-01-09 12:58:00,1980-01-10 22:25:00,1.39,0,1,68.006222,91.0,170.0,4,1,0,0,1,1,0,0,1,1,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,17.5,26.750000,20.750000,8.475000,96.500000,1.025000,110.000000,136.5,4.1000,40.050,13.125000,27.425,32.8750,83.250000,125.000000,4.85750,15.525,10.400000,1.500000,16.025000,43.125000


In [5]:
Cat0_train.dtypes

id                                                                   int64
hosp_admittime                                                      object
hosp_dischtime                                                      object
icu_intime                                                          object
icu_outtime                                                         object
los_icu                                                            float64
icu_death                                                            int64
gender                                                               int64
admission_age                                                      float64
weight_admit                                                       float64
height                                                             float64
charlson_score                                                       int64
atrial_fibrillation                                                  int64
malignant_cancer         

In [6]:
columns_to_drop = ['id',
                   #'charttime',
                   #'hour',
                   #'icu_cat',
                   'hosp_admittime',
                   'hosp_dischtime',
                   'icu_intime',
                   'icu_outtime',
                   'los_icu',
                   'icu_death',
                   #'race',
                   'icu_outcome'
                  ]

Cat0_X_train = Cat0_train.drop(columns=columns_to_drop)
Cat0_X_test = Cat0_test.drop(columns=columns_to_drop)
Cat1_X_train = Cat1_train.drop(columns=columns_to_drop)
Cat1_X_test = Cat1_test.drop(columns=columns_to_drop)

Cat0_y_train = Cat0_train.los_icu
Cat0_y_test = Cat0_test.los_icu
Cat1_y_train = Cat1_train.los_icu
Cat1_y_test = Cat1_test.los_icu

print(Cat0_X_train.shape)
print(Cat0_y_train.shape)
print(Cat0_X_test.shape)
print(Cat0_y_test.shape)
print(Cat1_X_train.shape)
print(Cat1_y_train.shape)
print(Cat1_X_test.shape)
print(Cat1_y_test.shape)
                   

(7136, 59)
(7136,)
(1783, 59)
(1783,)
(4766, 59)
(4766,)
(1193, 59)
(1193,)


In [7]:

# Perform normalization using data from X_train to transform X_test for CAT0

from sklearn.preprocessing import StandardScaler

num_cols = Cat0_X_train.columns[Cat0_X_train.dtypes.apply(lambda c: np.issubdtype(c, np.number))]
# print(num_cols)
scaler = StandardScaler()
Cat0_X_train[num_cols] = scaler.fit_transform(Cat0_X_train[num_cols])
Cat0_X_test[num_cols] = scaler.transform(Cat0_X_test[num_cols])

In [8]:
# Perform normalization using data from X_train to transform X_test for CAT1

from sklearn.preprocessing import StandardScaler

num_cols1 = Cat1_X_train.columns[Cat1_X_train.dtypes.apply(lambda c: np.issubdtype(c, np.number))]
# print(num_cols1)
Scaler1 = StandardScaler()
Cat1_X_train[num_cols1] = Scaler1.fit_transform(Cat1_X_train[num_cols1])
Cat1_X_test[num_cols1] = Scaler1.transform(Cat1_X_test[num_cols1])

In [9]:
Cat1_X_test.describe()

,gender,admission_age,weight_admit,height,charlson_score,atrial_fibrillation,malignant_cancer,chf,ckd,cld,copd,diabetes,hypertension,ihd,stroke,race_encode_African,race_encode_Asian,race_encode_Caucasian,race_encode_Hispanic,race_encode_Not Specified,race_encode_South American,admission_type_DIRECT EMER.,admission_type_DIRECT OBSERVATION,admission_type_ELECTIVE,admission_type_EU OBSERVATION,admission_type_EW EMER.,admission_type_OBSERVATION ADMIT,admission_type_SURGICAL SAME DAY ADMISSION,admission_type_URGENT,first_careunit_Cardiac Vascular Intensive Care Unit (CVICU),first_careunit_Coronary Care Unit (CCU),first_careunit_Medical Intensive Care Unit (MICU),first_careunit_Medical/Surgical Intensive Care Unit (MICU/SICU),first_careunit_Neuro Intermediate,first_careunit_Neuro Stepdown,first_careunit_Neuro Surgical Intensive Care Unit (Neuro SICU),first_careunit_Surgical Intensive Care Unit (SICU),first_careunit_Trauma SICU (TSICU),aniongap,bicarbonate,bun,calcium,chloride,creatinine,glucose,sodium,potassium,hematocrit,hemoglobin,mch,mchc,mcv,platelet,rbc,rdw,wbc,inr,pt,ptt
count,1193.000000,1193.000000,1193.000000,1193.000000,1193.000000,1193.000000,1193.000000,1193.000000,1193.000000,1193.000000,1193.000000,1193.000000,1193.000000,1193.000000,1193.000000,1193.000000,1193.000000,1193.000000,1193.000000,1193.000000,1193.000000,1193.000000,1193.0,1193.000000,1193.0,1193.000000,1193.000000,1193.000000,1193.000000,1193.000000,1193.000000,1193.000000,1193.000000,1193.000000,1193.000000,1193.000000,1193.000000,1193.000000,1193.000000,1193.000000,1193.000000,1193.000000,1193.000000,1193.000000,1193.000000,1193.000000,1193.000000,1193.000000,1193.000000,1193.000000,1193.000000,1193.000000,1193.000000,1193.000000,1193.000000,1193.000000,1193.000000,1193.000000,1193.000000
mean,-0.051625,-0.046835,-0.088231,-0.006217,-0.060665,-0.035269,-0.030438,-0.045936,-0.033636,0.119137,0.020709,-0.060989,-0.050631,0.003407,-0.026730,0.002621,0.010872,0.000434,0.009783,-0.018174,0.057856,-0.015418,0.0,-0.048387,0.0,0.034419,0.002641,-0.037258,0.005909,-0.002453,0.042526,-0.035204,0.012244,-0.045378,-0.020155,0.017247,0.013413,-0.012869,-0.041731,0.024899,-0.084006,-0.003505,0.034170,-0.089561,0.002605,0.023085,-0.052020,-0.004817,-0.017478,-0.017472,-0.056307,0.010366,0.070990,-0.001690,0.010053,-0.010969,-0.018893,-0.024856,-0.007250
std,1.005098,1.024614,0.854246,0.925715,0.984555,0.981210,0.967098,0.985749,0.995601,1.137393,1.010360,0.976402,1.024951,1.000554,0.967236,1.003284,1.034008,1.000261,1.024197,0.980870,1.730235,0.965898,0.0,0.849712,0.0,0.996274,1.003336,0.930620,1.005238,0.997348,1.045347,0.981419,1.010036,0.749920,0.784713,1.061280,1.013141,0.984820,0.942935,0.980695,0.936200,1.007924,1.076381,0.807803,0.913306,1.178180,0.978891,0.983116,0.980150,1.035589,1.015769,1.051519,1.099585,1.023055,0.981947,0.660096,0.910949,0.839368,0.968503
min,-1.123381,-3.189145,-3.094076,-4.759038,-2.033770,-0.606280,-0.397038,-0.748375,-0.881870,-0.342736,-0.622156,-0.703882,-1.640644,-0.959687,-0.360443,-0.387781,-0.155130,-1.428804,-0.195238,-0.398453,-0.028982,-0.217353,0.0,-0.170086,0.0,-1.108217,-0.384912,-0.260560,-0.473217,-0.351087,-0.386348,-0.608869,-0.482492,-0.102967,-0.052298,-0.134754,-0.425978,-0.363039,-2.428233,-3.275064,-1.240372,-3.936922,-3.860509,-0.844332,-1.819574,-6.148823,-3.057700,-2.905165,-2.730234,-5.206527,-4.290759,-4.457897,-1.984688,-3.075383,-2.070766,-1.286351,-0.995997,-0.985860,-1.420820
25%,-1.123381,-0.672128,-0.666987,-0.076235,-0.667797,-0.606280,-0.397038,-0.748375,-0.881870,-0.342736,-0.622156,-0.703882,-1.640644,-0.959687,-0.360443,-0.387781,-0.155130,-1.428804,-0.195238,-0.398453,-0.028982,-0.217353,0.0,-0.170086,0.0,-1.108217,-0.384912,-0.260560,-0.473217,-0.351087,-0.386348,-0.608869,-0.482492,-0.102967,-0.052298,-0.134754,-0.425978,-0.363039,-0.609355,-0.617084,-0.691435,-0.548169,-0.636359,-0.540258,-0.566190,-0.505710,-0.755386,-0.684289,-0.689520,-0.475233,-0.655353,-0.520464,-0.552978,-0.665969,-0.61417

In [10]:
print(Cat0_X_train.dtypes)

gender                                                             float64
admission_age                                                      float64
weight_admit                                                       float64
height                                                             float64
charlson_score                                                     float64
atrial_fibrillation                                                float64
malignant_cancer                                                   float64
chf                                                                float64
ckd                                                                float64
cld                                                                float64
copd                                                               float64
diabetes                                                           float64
hypertension                                                       float64
ihd                      

In [11]:
import torch
import torch.nn as nn
import torch.optim as optim

In [12]:
Cat0_X_train_tensor = torch.tensor(Cat0_X_train.to_numpy(),dtype=torch.float32)
m,n = Cat0_X_train_tensor.shape
Cat0_y_train_tensor = torch.tensor(Cat0_y_train.to_numpy(),dtype=torch.float32).reshape(m,1)

Cat0_X_test_tensor = torch.tensor(Cat0_X_test.to_numpy(),dtype=torch.float32)
m,n = Cat0_X_test_tensor.shape
Cat0_y_test_tensor = torch.tensor(Cat0_y_test.to_numpy(),dtype=torch.float32).reshape(m,1)

Cat1_X_train_tensor = torch.tensor(Cat1_X_train.to_numpy(),dtype=torch.float32)
m,n = Cat1_X_train_tensor.shape
Cat1_y_train_tensor = torch.tensor(Cat1_y_train.to_numpy(),dtype=torch.float32).reshape(m,1)

Cat1_X_test_tensor = torch.tensor(Cat1_X_test.to_numpy(),dtype=torch.float32)
m,n = Cat1_X_test_tensor.shape
Cat1_y_test_tensor = torch.tensor(Cat1_y_test.to_numpy(),dtype=torch.float32).reshape(m,1)

In [17]:
# CAT0 NN model

class NN_Regressor(nn.Module):
    def __init__(self, input_size, output_size, dropout_prob): # set the arguments you'd need, including activation function
        super(NN_Regressor, self).__init__()
        self.input_size = input_size
        self.output_size = output_size

        self.layer1 = nn.Linear(self.input_size, 128)
        self.hidden2 = nn.Linear(128, 128)
        self.hidden3 = nn.Linear(128, 32)
        # self.hidden4 = nn.Linear(32, 16)
        # self.hidden5 = nn.Linear(16, 16)
        # self.hidden6 = nn.Linear(16, 8)
        self.output7 = nn.Linear(32, output_size)

        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout_prob)

    def forward(self, x):
        x = self.layer1(x)
        x = self.relu(x)
        x = self.dropout(x)
        
        x = self.hidden2(x)
        x = self.relu(x)
        x = self.dropout(x)

        x = self.hidden3(x)
        x = self.relu(x)
        x = self.dropout(x)

        # x = self.hidden4(x)
        # x = self.relu(x)
        # x = self.dropout(x)

        # x = self.hidden5(x)
        # x = self.relu(x)
        # x = self.dropout(x)

        # x = self.hidden6(x)
        # x = self.relu(x)
        # x = self.dropout(x)

        x = self.output7(x)
        # x = self.relu(x)

        return x


def train_model(x_train, y_train, epochs=400):
    model = NN_Regressor(59, 1, 0.3)
    optimiser = optim.Adam(model.parameters(), lr=0.001)
    loss_fn = nn.MSELoss() 

    for i in range(epochs):
        # reset gradients to 0
        optimiser.zero_grad()

        # get predictions
        y_pred = model.forward(x_train)
        
        # compute loss (uncomment the next line and fill right hand side)
        abs_loss = loss_fn(y_pred, y_train) 

        # backpropagate
        abs_loss.backward()
    
        # update the model weights
        optimiser.step()
        
        print (f"{i:5d}", abs_loss.item(), sep='\t')
        
    return model
                

NNRegressor = train_model(Cat0_X_train_tensor, Cat0_y_train_tensor)

    0	2.618826389312744
    1	2.4132561683654785
    2	2.224301815032959
    3	2.0445284843444824
    4	1.8772059679031372
    5	1.7063130140304565
    6	1.5331075191497803
    7	1.3645163774490356
    8	1.2017408609390259
    9	1.0354465246200562
   10	0.8829059600830078
   11	0.752891480922699
   12	0.6163167357444763
   13	0.5134074687957764
   14	0.47490838170051575
   15	0.45151394605636597
   16	0.49547848105430603
   17	0.5237748026847839
   18	0.5884053111076355
   19	0.5833721160888672
   20	0.5824755430221558
   21	0.5779179334640503
   22	0.5274084806442261
   23	0.47740620374679565
   24	0.4569063186645508
   25	0.4050227403640747
   26	0.3880956470966339
   27	0.4001415967941284
   28	0.3949733078479767
   29	0.382845401763916
   30	0.39650997519493103
   31	0.41314005851745605
   32	0.4110538065433502
   33	0.4002441465854645
   34	0.40815383195877075
   35	0.4033505916595459
   36	0.391266405582428
   37	0.38104379177093506
   38	0.3790256381034851
   39	0.36357483267784

In [18]:
# checking MSE on testing set of Cat0 regression model
from sklearn.metrics import mean_squared_error

NNRegressor.eval()
with torch.no_grad():
    y_test_pred = NNRegressor(Cat0_X_test_tensor)

mse_test_Cat0_NNregressor = mean_squared_error(Cat0_y_test_tensor, y_test_pred)
print("Cat0 NN regressor y_test MSE:", mse_test_Cat0_NNregressor)



with torch.no_grad():
    y_train_pred = NNRegressor(Cat0_X_train_tensor)

mse_train_Cat0_NNregressor = mean_squared_error(Cat0_y_train_tensor, y_train_pred)
print("Cat0 NN regressor y_train MSE:", mse_train_Cat0_NNregressor)

Cat0 NN regressor y_test MSE: 0.19935486
Cat0 NN regressor y_train MSE: 0.1597823


In [35]:
# CAT1 NN model

class NN_Regressor_cat1(nn.Module):
    def __init__(self, input_size, output_size, dropout_prob): # set the arguments you'd need, including activation function
        super(NN_Regressor_cat1, self).__init__()
        self.input_size = input_size
        self.output_size = output_size

        self.layer1 = nn.Linear(self.input_size, 64)
        self.hidden2 = nn.Linear(64, 64)
        self.hidden3 = nn.Linear(64, 16)
        # self.hidden4 = nn.Linear(32, 16)
        # self.hidden5 = nn.Linear(16, 16)
        # self.hidden6 = nn.Linear(16, 8)
        self.output7 = nn.Linear(16, output_size)

        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout_prob)

    def forward(self, x):
        x = self.layer1(x)
        x = self.relu(x)
        x = self.dropout(x)
        
        x = self.hidden2(x)
        x = self.relu(x)
        x = self.dropout(x)

        x = self.hidden3(x)
        x = self.relu(x)
        x = self.dropout(x)

        # x = self.hidden4(x)
        # x = self.relu(x)
        # x = self.dropout(x)

        # x = self.hidden5(x)
        # x = self.relu(x)
        # x = self.dropout(x)

        # x = self.hidden6(x)
        # x = self.relu(x)
        # x = self.dropout(x)

        x = self.output7(x)
        # x = self.relu(x)

        return x


def train_model(x_train, y_train, epochs=500):
    model = NN_Regressor_cat1(59, 1, 0.3)
    optimiser = optim.Adam(model.parameters(), lr=0.002)
    loss_fn = nn.MSELoss() 

    for i in range(epochs):
        # reset gradients to 0
        optimiser.zero_grad()

        # get predictions
        y_pred = model.forward(x_train)
        
        # compute loss (uncomment the next line and fill right hand side)
        abs_loss = loss_fn(y_pred, y_train) 

        # backpropagate
        abs_loss.backward()
    
        # update the model weights
        optimiser.step()
        
        print (f"{i:5d}", abs_loss.item(), sep='\t')
        
    return model
                

NNRegressor_cat1 = train_model(Cat1_X_train_tensor, Cat1_y_train_tensor)

    0	12.902070999145508
    1	12.590081214904785
    2	12.28461742401123
    3	11.974842071533203
    4	11.649019241333008
    5	11.328559875488281
    6	10.95574951171875
    7	10.5209321975708
    8	10.053547859191895
    9	9.58351993560791
   10	9.060052871704102
   11	8.506672859191895
   12	7.871405124664307
   13	7.1521100997924805
   14	6.5328545570373535
   15	5.8280110359191895
   16	5.064774990081787
   17	4.371153354644775
   18	3.7746262550354004
   19	3.195220947265625
   20	2.6518735885620117
   21	2.4256677627563477
   22	2.4153594970703125
   23	2.4994328022003174
   24	2.8079493045806885
   25	2.8608591556549072
   26	3.3109710216522217
   27	3.002330780029297
   28	2.8912947177886963
   29	2.683089017868042
   30	2.4110140800476074
   31	2.2615532875061035
   32	2.0832648277282715
   33	2.089557409286499
   34	2.045586585998535
   35	2.0525624752044678
   36	2.0391485691070557
   37	2.0234556198120117
   38	2.114685535430908
   39	2.097076416015625
   40	2.1020576953

In [36]:
# checking MSE on testing set of Cat1 regression model
from sklearn.metrics import mean_squared_error

NNRegressor_cat1.eval()
with torch.no_grad():
    y_test_pred = NNRegressor_cat1(Cat1_X_test_tensor)

mse_test_Cat1_NNregressor = mean_squared_error(Cat1_y_test_tensor, y_test_pred)
print("Cat1 NN regressor y_test MSE:", mse_test_Cat1_NNregressor)



with torch.no_grad():
    y_train_pred = NNRegressor_cat1(Cat1_X_train_tensor)

mse_train_Cat1_NNregressor = mean_squared_error(Cat1_y_train_tensor, y_train_pred)
print("Cat1 NN regressor y_train MSE:", mse_train_Cat1_NNregressor)

Cat1 NN regressor y_test MSE: 0.5339166
Cat1 NN regressor y_train MSE: 0.47028637


In [5]:
# getting number of unique patient ID
print(static_df.shape)
static_df.id.unique

(20414, 25)


<bound method Series.unique of 0        24099382
1        27824879
2        27146619
3        22476441
4        23169158
           ...   
20409    29968077
20410    29975784
20411    29987115
20412    29990494
20413    29994623
Name: id, Length: 20414, dtype: int64>

In [6]:
from sklearn.model_selection import train_test_split

#splitting the data into 70 20 10 based on unique ID

XY_features = ['id', 'los_icu', 'icu_death']
static_split = static_df[XY_features]

X = static_split.id
y = static_split.los_icu

# Split dataset into training set, test set, and holdout set

X_train_temp, X_holdout = train_test_split(static_split, test_size=0.1, random_state=42)
X_train, X_test = train_test_split(X_train_temp, test_size=0.22222, random_state=42)

print(X_train.shape)
print(X_test.shape)
print(X_holdout.shape)

(14289, 3)
(4083, 3)
(2042, 3)


In [7]:
# Getting the different unique id from training, testing and holdout data

train_id = X_train.id.unique().tolist()
test_id = X_test.id.unique().tolist()
holdout_id = X_holdout.id.unique().tolist()

print(len(train_id))
print(len(test_id))
print(len(holdout_id))

14289
4083
2042


In [8]:
# encoding static_df female to 0, male to 1

static_df.loc[static_df.gender == 'F', 'gender'] = 0
static_df.loc[static_df.gender == 'M', 'gender'] = 1



In [9]:
# Reducing number of race
static_df[["race_encode"]] = 'placeholder'
static_df.loc[static_df.race.astype(str).str.lower().str.contains('asian|indian'), "race_encode"] = 'Asian'
static_df.loc[static_df.race.astype(str).str.lower().str.contains('black'), "race_encode"] = 'African'
static_df.loc[static_df.race.astype(str).str.lower().str.contains('white|portuguese'), "race_encode"] = 'Caucasian'
static_df.loc[static_df.race.astype(str).str.lower().str.contains('hispanic'), "race_encode"] = 'Hispanic'
static_df.loc[static_df.race.astype(str).str.lower().str.contains('other|multiple|unknown|declined|unable'), "race_encode"] = 'Not Specified'
static_df.loc[static_df.race.astype(str).str.lower().str.contains('south american'), "race_encode"] = 'South American'

In [10]:
# smoothing icu_los followed by encoding ICU outcome
# los_icu 25th percentile = 1.8, 50th = 2.88, 75th = 5.28


static_df[['icu_outcome']] = 'placeholder'
static_df.loc[(static_df['los_icu'] <= 1.8) & (static_df['icu_death'] == 0), 'icu_outcome'] = 1
static_df.loc[(static_df['los_icu'] > 1.8) & (static_df['los_icu'] <= 2.88) & (static_df['icu_death'] == 0), 'icu_outcome'] = 2
static_df.loc[(static_df['los_icu'] > 2.88) & (static_df['los_icu'] <= 5.28) & (static_df['icu_death'] == 0), 'icu_outcome'] = 3
static_df.loc[(static_df['los_icu'] > 5.28) & (static_df['icu_death'] == 0), 'icu_outcome'] = 4
static_df.loc[(static_df['los_icu'] <= 1.8) & (static_df['icu_death'] == 1), 'icu_outcome'] = 5
static_df.loc[(static_df['los_icu'] > 1.8) & (static_df['los_icu'] <= 2.88) & (static_df['icu_death'] == 1), 'icu_outcome'] = 6
static_df.loc[(static_df['los_icu'] > 2.88) & (static_df['los_icu'] <= 5.28) & (static_df['icu_death'] == 1), 'icu_outcome'] = 7
static_df.loc[(static_df['los_icu'] > 5.28) & (static_df['icu_death'] == 1), 'icu_outcome'] = 8

static_df.head()

,id,hosp_admittime,hosp_dischtime,icu_intime,icu_outtime,los_icu,icu_death,gender,race,admission_age,weight_admit,height,admission_type,first_careunit,charlson_score,atrial_fibrillation,malignant_cancer,chf,ckd,cld,copd,diabetes,hypertension,ihd,stroke,race_encode,icu_outcome
0,24099382,6/11/11 7:15,6/18/11 21:45,6/11/11 14:31,6/13/11 18:05,2.15,0,1,OTHER,58.441631,92.0,170.0,ELECTIVE,Trauma SICU (TSICU),11,0,0,0,0,0,1,1,1,0,0,Not Specified,2
1,27824879,9/30/28 20:55,10/10/28 15:54,9/30/28 20:58,10/5/28 18:39,4.90,0,0,WHITE,79.748598,55.0,NaN,EW EMER.,Trauma SICU (TSICU),5,0,0,0,0,0,1,0,1,1,0,Caucasian,3
2,27146619,9/20/41 0:34,9/22/41 12:05,9/20/41 7:13,9/21/41 18:10,1.46,0,0,WHITE,81.717397,59.0,NaN,EW EMER.,Medical/Surgical Intensive Care Unit (MICU/SICU),8,0,0,1,1,0,0,0,0,1,0,Caucasian,1
3,22476441,10/8/73 16:46,10/16/73 13:05,10/8/73 18:28,10/10/73 13:26,1.79,0,0,BLACK/AFRICAN AMERICAN,63.769278,73.6,160.0,EW EMER.,Medical/Surgical Intensive Care Unit (MICU/SICU),8,0,0,1,1,0,1,1,1,1,0,African,1
4,23169158,3/12/27 1:19,3/19/27 13:15,3/12/27 2:43,3/15/27 1:53,2.97,0,0,WHITE,84.191979,79.1,160.0,OBSERVATION ADMIT,Medical/Surgical Intensive Care Unit (MICU/SICU),6,1,0,1,1,0,0,1,1,0,0,Caucasian,3


In [11]:
print(static_df.shape)
static_df.race_encode.unique()

(20414, 27)


array(['Not Specified', 'Caucasian', 'African', 'Asian', 'Hispanic',
       'South American'], dtype=object)

In [12]:
# one-hot encoding of race (6), admission_type (8), first_careunit (9)
# expecting to see 27 - 1 + 6 - 1 + 8 - 1 + 9 = 47 features

static_df = pd.get_dummies(static_df, columns=['race_encode'])
static_df = pd.get_dummies(static_df, columns=['admission_type'])
static_df = pd.get_dummies(static_df, columns=['first_careunit'])
print(static_df.shape)
static_df.head()

(20414, 47)


,id,hosp_admittime,hosp_dischtime,icu_intime,icu_outtime,los_icu,icu_death,gender,race,admission_age,weight_admit,height,charlson_score,atrial_fibrillation,malignant_cancer,chf,ckd,cld,copd,diabetes,hypertension,ihd,stroke,icu_outcome,race_encode_African,race_encode_Asian,race_encode_Caucasian,race_encode_Hispanic,race_encode_Not Specified,race_encode_South American,admission_type_DIRECT EMER.,admission_type_DIRECT OBSERVATION,admission_type_ELECTIVE,admission_type_EU OBSERVATION,admission_type_EW EMER.,admission_type_OBSERVATION ADMIT,admission_type_SURGICAL SAME DAY ADMISSION,admission_type_URGENT,first_careunit_Cardiac Vascular Intensive Care Unit (CVICU),first_careunit_Coronary Care Unit (CCU),first_careunit_Medical Intensive Care Unit (MICU),first_careunit_Medical/Surgical Intensive Care Unit (MICU/SICU),first_careunit_Neuro Intermediate,first_careunit_Neuro Stepdown,first_careunit_Neuro Surgical Intensive Care Unit (Neuro SICU),first_careunit_Surgical Intensive Care Unit (SICU),first_careunit_Trauma SICU (TSICU)
0,24099382,6/11/11 7:15,6/18/11 21:45,6/11/11 14:31,6/13/11 18:05,2.15,0,1,OTHER,58.441631,92.0,170.0,11,0,0,0,0,0,1,1,1,0,0,2,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,27824879,9/30/28 20:55,10/10/28 15:54,9/30/28 20:58,10/5/28 18:39,4.90,0,0,WHITE,79.748598,55.0,NaN,5,0,0,0,0,0,1,0,1,1,0,3,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1
2,27146619,9/20/41 0:34,9/22/41 12:05,9/20/41 7:13,9/21/41 18:10,1.46,0,0,WHITE,81.717397,59.0,NaN,8,0,0,1,1,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0
3,22476441,10/8/73 16:46,10/16/73 13:05,10/8/73 18:28,10/10/73 13:26,1.79,0,0,BLACK/AFRICAN AMERICAN,63.769278,73.6,160.0,8,0,0,1,1,0,1,1,1,1,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0
4,23169158,3/12/27 1:19,3/19/27 13:15,3/12/27 2:43,3/15/27 1:53,2.97,0,0,WHITE,84.191979,79.1,160.0,6,1,0,1,1,0,0,1,1,0,0,3,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0


In [13]:
#imputing height and weight with median values

static_df['height'] = static_df['height'].fillna(static_df['height'].median())
static_df['weight_admit'] = static_df['weight_admit'].fillna(static_df['weight_admit'].median())
static_df.head()

,id,hosp_admittime,hosp_dischtime,icu_intime,icu_outtime,los_icu,icu_death,gender,race,admission_age,weight_admit,height,charlson_score,atrial_fibrillation,malignant_cancer,chf,ckd,cld,copd,diabetes,hypertension,ihd,stroke,icu_outcome,race_encode_African,race_encode_Asian,race_encode_Caucasian,race_encode_Hispanic,race_encode_Not Specified,race_encode_South American,admission_type_DIRECT EMER.,admission_type_DIRECT OBSERVATION,admission_type_ELECTIVE,admission_type_EU OBSERVATION,admission_type_EW EMER.,admission_type_OBSERVATION ADMIT,admission_type_SURGICAL SAME DAY ADMISSION,admission_type_URGENT,first_careunit_Cardiac Vascular Intensive Care Unit (CVICU),first_careunit_Coronary Care Unit (CCU),first_careunit_Medical Intensive Care Unit (MICU),first_careunit_Medical/Surgical Intensive Care Unit (MICU/SICU),first_careunit_Neuro Intermediate,first_careunit_Neuro Stepdown,first_careunit_Neuro Surgical Intensive Care Unit (Neuro SICU),first_careunit_Surgical Intensive Care Unit (SICU),first_careunit_Trauma SICU (TSICU)
0,24099382,6/11/11 7:15,6/18/11 21:45,6/11/11 14:31,6/13/11 18:05,2.15,0,1,OTHER,58.441631,92.0,170.0,11,0,0,0,0,0,1,1,1,0,0,2,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,27824879,9/30/28 20:55,10/10/28 15:54,9/30/28 20:58,10/5/28 18:39,4.90,0,0,WHITE,79.748598,55.0,170.0,5,0,0,0,0,0,1,0,1,1,0,3,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1
2,27146619,9/20/41 0:34,9/22/41 12:05,9/20/41 7:13,9/21/41 18:10,1.46,0,0,WHITE,81.717397,59.0,170.0,8,0,0,1,1,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0
3,22476441,10/8/73 16:46,10/16/73 13:05,10/8/73 18:28,10/10/73 13:26,1.79,0,0,BLACK/AFRICAN AMERICAN,63.769278,73.6,160.0,8,0,0,1,1,0,1,1,1,1,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0
4,23169158,3/12/27 1:19,3/19/27 13:15,3/12/27 2:43,3/15/27 1:53,2.97,0,0,WHITE,84.191979,79.1,160.0,6,1,0,1,1,0,0,1,1,0,0,3,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0


In [14]:
static_train_df = static_df[static_df['id'].isin(train_id)]
static_test_df = static_df[static_df['id'].isin(test_id)]
static_holdout_df = static_df[static_df['id'].isin(holdout_id)]
print(static_train_df.shape)
print(static_test_df.shape)
print(static_holdout_df.shape)

(14289, 47)
(4083, 47)
(2042, 47)


In [15]:
# merging dynamic data with encoded icu_outcome from static

static_icu_outcome = static_df[['id', 'icu_outcome']]

dynamic_df = pd.merge(dynamic_df, static_icu_outcome, on='id', how='left')

print(dynamic_df.shape)
dynamic_df.head()

(93604, 71)


,id,charttime,albumin,globulin,total_protein,aniongap,bicarbonate,bun,calcium,chloride,creatinine,glucose,sodium,potassium,heart_rate,sbp,dbp,mbp,resp_rate,temperature,spo2,hematocrit,hemoglobin,mch,mchc,mcv,platelet,rbc,rdw,wbc,alt,ast,alp,amylase,bilirubin_total,bilirubin_direct,bilirubin_indirect,ck_cpk,ck_mb,ggt,ld_ldh,so2,po2,pco2,fio2_chartevents,aado2,aado2_calc,pao2fio2ratio,ph,baseexcess,bicarbonate_bg,totalco2,hematocrit_bg,hemoglobin_bg,carboxyhemoglobin,methemoglobin,chloride_bg,calcium_bg,temperature_bg,potassium_bg,sodium_bg,lactate_bg,glucose_bg,d_dimer,fibrinogen,thrombin,inr,pt,ptt,urineoutput,icu_outcome
0,28793466,4/12/29 3:35,NaN,NaN,NaN,16.0,21.0,12.0,8.8,106.0,0.6,154.0,139.0,3.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30.0,9.6,30.3,32.0,95.0,256.0,3.17,12.5,12.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,10.9,27.8,NaN,4
1,25611175,11/29/40 3:50,NaN,NaN,NaN,15.0,27.0,28.0,8.5,101.0,1.7,223.0,139.0,3.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,28.0,9.1,30.3,32.5,93.0,216.0,3.00,13.8,6.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3
2,26115624,9/7/50 0:22,3.5,NaN,NaN,12.0,22.0,9.0,7.9,111.0,0.6,97.0,141.0,3.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26.4,9.6,31.3,36.5,86.0,115.0,3.07,15.4,0.8,18.0,17.0,141.0,NaN,0.8,NaN,NaN,NaN,NaN,NaN,155.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.2,13.9,25.8,NaN,2
3,28164589,3/11/59 1:11,NaN,NaN,NaN,12.0,34.0,58.0,8.3,103.0,2.1,89.0,146.0,3.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30.4,8.9,30.7,29.3,105.0,105.0,2.90,20.0,4.8,9.0,20.0,75.0,NaN,0.7,NaN,NaN,NaN,4.0,NaN,250.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.8,41.7,40.2,NaN,2
4,26115624,9/1/50 20:14,NaN,NaN,NaN,NaN,NaN,11.0,7.8,NaN,NaN,NaN,NaN,3.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,201.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2


# Imputation section

In [16]:
# # dropping features with > 50% missing values in dynamic_df
# missing_percentage = (dynamic_df.isna().sum() / len(dynamic_df)) * 100

# # Identify features with more than 50% missing values
# features_to_drop = missing_percentage[missing_percentage >= 50].index

# print(len(features_to_drop))

# dynamic_dropped = dynamic_df.drop(columns=features_to_drop)
# print(dynamic_df.shape)
# print(dynamic_dropped.shape)

In [17]:
# # Imputing missing values in dynamic_dropped with simple median values of each column

# dynamic_dropped = dynamic_dropped.fillna(dynamic_df.median())

In [18]:
# # Extract median imputation method to Excel for comparison of different imputation method

# dynamic_dropped.head()

# dynamic_df_median = dynamic_dropped.drop(columns=['icu_outcome'])
# print(dynamic_df_median.shape)

# dynamic_df_median.to_excel('dynamic_full_median.xlsx', index=False)

In [19]:
# # manually splitting dynamic_change_df into training, testing and holdout set based on previous split unique id above

# dynamic_train_df = dynamic_dropped[dynamic_dropped['id'].isin(train_id)]
# dynamic_test_df = dynamic_dropped[dynamic_dropped['id'].isin(test_id)]
# dynamic_holdout_df = dynamic_dropped[dynamic_dropped['id'].isin(holdout_id)]
# print(dynamic_train_df.shape)
# print(dynamic_test_df.shape)
# print(dynamic_holdout_df.shape)

In [20]:
# # Imputing remaining missing values with the median values from the same class label
# nan_counts_per_feature = static_df.isna().sum(axis=0)
# features_with_nan = nan_counts_per_feature[nan_counts_per_feature > 0].index.tolist()
# print("NaN counts for each feature:")
# print(nan_counts_per_feature)
# print(features_with_nan)

In [21]:
# # nan count for dynamic_dropped
# nan_counts_per_feature = dynamic_dropped.isna().sum(axis=0)
# features_with_nan = nan_counts_per_feature[nan_counts_per_feature > 0].index.tolist()
# print("NaN counts for each feature:")
# print(nan_counts_per_feature)
# print(features_with_nan)

In [22]:
# # imputing missing values based on median value of the same icu_outcome from the training dataset


# # Group the data by the class label
# grouped = dynamic_train_df.groupby('icu_outcome')

# grouped.describe()

# # nan count for dynamic_train_df
# nan_counts_per_feature = dynamic_train_df.isna().sum(axis=0)
# features_with_nan = nan_counts_per_feature[nan_counts_per_feature > 0].index.tolist()
# print("NaN counts for each feature:")
# print(nan_counts_per_feature)
# print(features_with_nan)


# # Iterate over each feature
# for feature in features_with_nan:  # Exclude the last column which is the class label
#     # Calculate the median value of the feature within each group
#     median_per_class = grouped[feature].median()
#     #print(grouped[feature].median())

    
#     # Replace missing values in each feature with the corresponding median value from the appropriate class label
#     # also replace missing values in test and holdout dataset with median values from the training dataset with the same 'icu_outcome'
#     for class_label, median_value in median_per_class.items():
#         dynamic_train_df.loc[(dynamic_train_df[feature].isnull()) & (dynamic_train_df['icu_outcome'] == class_label), feature] = median_value
#         dynamic_test_df.loc[(dynamic_test_df[feature].isnull()) & (dynamic_test_df['icu_outcome'] == class_label), feature] = median_value
#         dynamic_holdout_df.loc[(dynamic_holdout_df[feature].isnull()) & (dynamic_holdout_df['icu_outcome'] == class_label), feature] = median_value

In [23]:
# nan_counts_per_feature = dynamic_test_df.isna().sum(axis=0)
# features_with_nan = nan_counts_per_feature[nan_counts_per_feature > 0].index.tolist()
# print("NaN counts for each feature in test set:")
# print(nan_counts_per_feature)
# print(features_with_nan)

# nan_counts_per_feature = dynamic_holdout_df.isna().sum(axis=0)
# features_with_nan = nan_counts_per_feature[nan_counts_per_feature > 0].index.tolist()
# print("NaN counts for each feature in holdout set:")
# print(nan_counts_per_feature)
# print(features_with_nan)

In [24]:
# nan_counts_per_feature = dynamic_train_df.isna().sum(axis=0)
# features_with_nan = nan_counts_per_feature[nan_counts_per_feature > 0].index.tolist()
# print("NaN counts for each feature in training dataset:")
# print(nan_counts_per_feature)
# print(features_with_nan)

In [25]:
# # combining dynamic train, test and holdout into a single dynamic_imputed_df

# dynamic_imputed_df = pd.concat([dynamic_train_df, dynamic_test_df, dynamic_holdout_df], axis=0)
# dynamic_imputed_df.reset_index(drop=True, inplace=True)
# dynamic_imputed_df.shape

# Start of combining static and dynamic data - testing of different imputation method

In [17]:
# Uncomment the individual imputation method to test

# KNN imputation
dynamic_imputed_df = pd.read_csv('dynamic_full_KNN.csv')

# Mice imputation
#dynamic_imputed_df = pd.read_csv('dynamic_full_MICE_amendedcharttime.csv')

# Simple median imputation
#dynamic_imputed_df = pd.read_excel('dynamic_full_median.xlsx')

In [18]:
nan_counts_per_feature = dynamic_imputed_df.isna().sum(axis=0)
features_with_nan = nan_counts_per_feature[nan_counts_per_feature > 0].index.tolist()
print("NaN counts for each feature in dynamic_imputed dataset:")
print(nan_counts_per_feature)
print(features_with_nan)

NaN counts for each feature in dynamic_imputed dataset:
id             0
charttime      0
aniongap       0
bicarbonate    0
bun            0
calcium        0
chloride       0
creatinine     0
glucose        0
sodium         0
potassium      0
hematocrit     0
hemoglobin     0
mch            0
mchc           0
mcv            0
platelet       0
rbc            0
rdw            0
wbc            0
inr            0
pt             0
ptt            0
dtype: int64
[]


In [19]:
# finding out the first and last dynamic measurement for each patient ID

from datetime import datetime

def hours_difference(date_string):
    date_obj = datetime.strptime(date_string, '%m/%d/%y %H:%M')
    current_year = datetime.now().year
    if date_obj.year > current_year:
        date_obj = date_obj.replace(year=date_obj.year - 100)
    current_time = datetime.now()
    time_difference = current_time - date_obj
    hours_difference = time_difference.total_seconds() / 3600

    return hours_difference

dynamic_imputed_df['charttime_hours'] = dynamic_imputed_df['charttime'].apply(lambda x: hours_difference(x))

dynamic_imputed_df.head()

,id,charttime,aniongap,bicarbonate,bun,calcium,chloride,creatinine,glucose,sodium,potassium,hematocrit,hemoglobin,mch,mchc,mcv,platelet,rbc,rdw,wbc,inr,pt,ptt,charttime_hours
0,28793466,4/12/29 3:35,16.0,21.0,12.0,8.8,106.0,0.60,154.0,139.0,3.8,30.00,9.60,30.30,32.0,95.0,256.0,3.170,12.50,12.10,1.00,10.90,27.80,832604.036423
1,25611175,11/29/40 3:50,15.0,27.0,28.0,8.5,101.0,1.70,223.0,139.0,3.5,28.00,9.10,30.30,32.5,93.0,216.0,3.000,13.80,6.40,1.76,19.06,82.60,730627.786423
2,26115624,9/7/50 0:22,12.0,22.0,9.0,7.9,111.0,0.60,97.0,141.0,3.6,26.40,9.60,31.30,36.5,86.0,115.0,3.070,15.40,0.80,1.20,13.90,25.80,644975.253089
3,28164589,3/11/59 1:11,12.0,34.0,58.0,8.3,103.0,2.10,89.0,146.0,3.4,30.40,8.90,30.70,29.3,105.0,105.0,2.900,20.00,4.80,3.80,41.70,40.20,570406.436423
4,26115624,9/1/50 20:14,10.2,23.2,11.0,7.8,102.2,0.76,144.4,145.4,3.3,32.84,9.54,31.38,34.0,88.8,106.6,3.492,14.84,31.26,1.26,14.00,45.08,645099.386423


In [20]:
def get_last_measurement_index(group):
    last_measurement = group['charttime_hours'].min()
    last_measurement_index = group[group['charttime_hours'] == last_measurement].index[-1]
    return last_measurement_index

def get_first_measurement_index(group):
    first_measurement = group['charttime_hours'].max()
    first_measurement_index = group[group['charttime_hours'] == first_measurement].index[-1]
    return first_measurement_index

last_measurement_indices = dynamic_imputed_df.groupby('id').apply(get_last_measurement_index)
first_measurement_indices = dynamic_imputed_df.groupby('id').apply(get_first_measurement_index)

last_measurement_indices_list = last_measurement_indices.tolist()
first_measurement_indices_list = first_measurement_indices.tolist()

print(last_measurement_indices_list)
print(first_measurement_indices_list)

[11891, 86305, 26439, 88705, 22553, 49127, 26157, 1360, 91852, 92092, 14809, 64395, 10320, 15327, 45428, 51866, 7517, 70299, 65990, 34364, 6630, 58619, 41272, 8821, 56354, 65092, 186, 26218, 31277, 21526, 17914, 8519, 21373, 67164, 16005, 67223, 17877, 22011, 87837, 58302, 89960, 80569, 31758, 36888, 89034, 54185, 78633, 55303, 58945, 66982, 45255, 67222, 66159, 36427, 45260, 42030, 71035, 19207, 35016, 21071, 66812, 61923, 50315, 46415, 13961, 91750, 26868, 16384, 58881, 75611, 33574, 67904, 2898, 71199, 54955, 61682, 15426, 55679, 17940, 90505, 26635, 29774, 2129, 55508, 45474, 54078, 44830, 24804, 79705, 27128, 38628, 56460, 21274, 60746, 70217, 71256, 12458, 83512, 22424, 46076, 71284, 83659, 77420, 87085, 85667, 40575, 58779, 91004, 15939, 63984, 45922, 56793, 32000, 65487, 81, 89961, 29937, 75702, 15736, 35494, 39818, 55488, 93041, 34375, 66053, 12358, 30111, 32044, 36126, 82195, 58639, 87063, 65030, 41795, 80020, 11158, 30581, 56317, 58796, 50424, 77145, 38967, 22551, 51654, 861

In [21]:
set_last = set(last_measurement_indices_list)
set_first = set(first_measurement_indices_list)
index_with_1_measurement = set_last.intersection(set_first)

print(index_with_1_measurement)
print(len(index_with_1_measurement))

{1, 24578, 24583, 8, 9, 24585, 57354, 73739, 16399, 49171, 40984, 65563, 24604, 81951, 49185, 65573, 39, 40, 73769, 65578, 24623, 90159, 81971, 53, 57397, 90169, 49213, 32833, 8260, 57413, 81988, 90183, 16457, 41035, 73803, 57423, 65620, 85, 57444, 82023, 24681, 49259, 73840, 49267, 82036, 8310, 32887, 65654, 49274, 82045, 65662, 65664, 16513, 8323, 41092, 90253, 32911, 16529, 32913, 82065, 148, 8345, 65690, 90268, 32926, 73888, 8355, 90277, 73896, 8362, 73900, 32941, 57531, 73915, 8381, 90299, 8385, 32963, 65732, 41158, 41165, 206, 57551, 24786, 49364, 82133, 218, 33008, 49396, 73972, 73976, 24825, 256, 82193, 8471, 41241, 16675, 57637, 82214, 74023, 33069, 57646, 82226, 307, 33083, 33086, 321, 8513, 324, 41289, 90444, 24915, 33108, 82260, 90454, 82267, 49500, 353, 65890, 90465, 82279, 16744, 57707, 8556, 57710, 24949, 57718, 82301, 57726, 49537, 33154, 65921, 74115, 90505, 41359, 8592, 57743, 41362, 16787, 49555, 82319, 90514, 74135, 33177, 57759, 65954, 16808, 8618, 49580, 90543, 82

In [22]:
dynamic_df_last = dynamic_imputed_df.iloc[last_measurement_indices_list]
print(dynamic_df_last.id.nunique())

dynamic_df_first = dynamic_imputed_df.iloc[first_measurement_indices_list]
print(dynamic_df_first.id.nunique())

20414
20414


In [23]:
# Merge the two DataFrames based on the ID column
merged_dynamic_df = pd.merge(dynamic_df_last, dynamic_df_first, on='id', suffixes=('_last', '_first'))

# merged_dynamic_df.info()

dynamic_change_df = merged_dynamic_df.copy()


for column in dynamic_df_last.columns:
    if column != 'id' and column != 'charttime':
        dynamic_change_df[column + '_delta'] = merged_dynamic_df[column + '_last'] - merged_dynamic_df[column + '_first']
        # change_df.drop(column + '_first')

# Droping the measurement from the first reading
dynamic_change_df.drop(columns=[column + '_first' for column in dynamic_df_first.columns if column not in ['id', 'charttime']], inplace=True)

In [24]:
print(dynamic_change_df.columns.nunique())
dynamic_change_df.head()

47


,id,charttime_last,aniongap_last,bicarbonate_last,bun_last,calcium_last,chloride_last,creatinine_last,glucose_last,sodium_last,potassium_last,hematocrit_last,hemoglobin_last,mch_last,mchc_last,mcv_last,platelet_last,rbc_last,rdw_last,wbc_last,inr_last,pt_last,ptt_last,charttime_hours_last,charttime_first,aniongap_delta,bicarbonate_delta,bun_delta,calcium_delta,chloride_delta,creatinine_delta,glucose_delta,sodium_delta,potassium_delta,hematocrit_delta,hemoglobin_delta,mch_delta,mchc_delta,mcv_delta,platelet_delta,rbc_delta,rdw_delta,wbc_delta,inr_delta,pt_delta,ptt_delta,charttime_hours_delta
0,20001305,3/25/78 21:55,13.0,24.0,50.0,10.8,108.0,0.9,131.0,141.0,4.1,32.66,9.20,28.98,33.20,87.6,136.6,3.748,15.90,12.46,1.30,14.10,33.10,403489.703119,3/25/78 8:20,-2.0,1.0,3.0,-0.6,0.0,0.1,-23.0,-1.0,-0.2,0.0,-0.58,-0.2,0.22,1.6,-104.4,-0.020,1.00,-3.36,-0.18,-2.44,-3.80,-13.583354
1,20001361,5/5/43 15:02,18.0,23.0,40.0,7.9,107.0,3.7,123.0,144.0,3.8,32.20,9.24,28.70,32.14,89.2,126.8,3.688,16.02,13.72,1.60,17.70,30.00,709328.586585,5/4/43 17:24,4.0,1.0,12.0,1.6,0.0,1.2,-38.0,7.0,-2.0,-1.7,-2.36,-2.0,-1.96,-0.8,-89.2,-0.072,2.92,-7.08,-0.20,-1.70,-3.50,-21.633297
2,20001770,1/26/17 3:56,14.0,22.0,23.0,7.4,86.0,1.3,80.0,122.0,4.5,27.90,9.80,34.60,35.10,99.0,173.0,2.830,20.20,13.80,1.70,18.50,49.80,63019.686478,1/26/17 3:56,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.000,0.00,0.00,0.00,0.00,0.00,0.000000
3,20002506,3/20/32 1:01,14.0,22.0,9.0,8.8,108.0,0.7,94.0,144.0,4.0,37.80,12.60,30.10,33.30,90.0,175.0,4.180,12.00,8.30,1.20,13.20,30.90,806854.603256,3/19/32 6:13,-2.0,0.0,-4.0,-0.5,7.0,0.0,-6.0,5.0,-0.1,-2.2,-1.20,0.0,-1.20,3.0,-48.0,-0.400,0.20,-2.40,0.00,0.60,2.20,-18.799950
4,20003425,7/23/55 2:19,11.0,21.0,23.0,8.5,101.0,0.8,162.0,133.0,5.0,24.40,7.60,28.00,31.10,90.0,211.0,2.710,15.70,10.00,2.06,20.52,62.06,602253.303138,7/21/55 23:27,3.0,-8.0,-3.0,-0.9,4.0,0.0,45.0,-1.0,0.0,-4.2,-1.30,-0.2,0.00,-1.0,-23.0,-0.450,0.20,4.40,0.66,5.82,26.46,-26.866761


In [25]:
print(static_df.columns.nunique())
print(dynamic_change_df.columns.nunique())

47
47


In [26]:
static_df_dropped = static_df.drop(columns=['hosp_admittime',
                                            'hosp_dischtime',
                                            'icu_intime',
                                            'icu_outtime',
                                            'race',
                                            'icu_outcome'
                                            ])
print(static_df_dropped.columns.nunique())


dynamic_df_dropped = dynamic_change_df.drop(columns=['charttime_last',
                                                    #'icu_outcome_last',
                                                    'charttime_hours_last',
                                                    'charttime_first',
                                                    #'icu_outcome_delta',
                                                    'charttime_hours_delta'
                                                    ])
print(dynamic_df_dropped.columns.nunique())

41
43


In [27]:
# combining both static and dynamic df into a single df
combined_df = pd.merge(static_df_dropped, dynamic_df_dropped, on='id', how='left')

combined_df.shape

(20414, 83)

In [28]:
# splitting pre-processed data back into train, test, holdout based on patient id
Train = combined_df[combined_df['id'].isin(train_id)]
Test = combined_df[combined_df['id'].isin(test_id)]
Holdout = combined_df[combined_df['id'].isin(holdout_id)]

In [29]:
X_features_to_drop = ['id', 'los_icu', 'icu_death']

X_train = Train.drop(columns=X_features_to_drop)
y_train = Train.los_icu

X_test = Test.drop(columns=X_features_to_drop)
y_test = Test.los_icu

X_holdout = Holdout.drop(columns=X_features_to_drop)
y_holdout = Holdout.los_icu

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)
print(X_holdout.shape)
print(y_holdout.shape)
X_train.head()

(14289, 80)
(14289,)
(4083, 80)
(4083,)
(2042, 80)
(2042,)


,gender,admission_age,weight_admit,height,charlson_score,atrial_fibrillation,malignant_cancer,chf,ckd,cld,copd,diabetes,hypertension,ihd,stroke,race_encode_African,race_encode_Asian,race_encode_Caucasian,race_encode_Hispanic,race_encode_Not Specified,race_encode_South American,admission_type_DIRECT EMER.,admission_type_DIRECT OBSERVATION,admission_type_ELECTIVE,admission_type_EU OBSERVATION,admission_type_EW EMER.,admission_type_OBSERVATION ADMIT,admission_type_SURGICAL SAME DAY ADMISSION,admission_type_URGENT,first_careunit_Cardiac Vascular Intensive Care Unit (CVICU),first_careunit_Coronary Care Unit (CCU),first_careunit_Medical Intensive Care Unit (MICU),first_careunit_Medical/Surgical Intensive Care Unit (MICU/SICU),first_careunit_Neuro Intermediate,first_careunit_Neuro Stepdown,first_careunit_Neuro Surgical Intensive Care Unit (Neuro SICU),first_careunit_Surgical Intensive Care Unit (SICU),first_careunit_Trauma SICU (TSICU),aniongap_last,bicarbonate_last,bun_last,calcium_last,chloride_last,creatinine_last,glucose_last,sodium_last,potassium_last,hematocrit_last,hemoglobin_last,mch_last,mchc_last,mcv_last,platelet_last,rbc_last,rdw_last,wbc_last,inr_last,pt_last,ptt_last,aniongap_delta,bicarbonate_delta,bun_delta,calcium_delta,chloride_delta,creatinine_delta,glucose_delta,sodium_delta,potassium_delta,hematocrit_delta,hemoglobin_delta,mch_delta,mchc_delta,mcv_delta,platelet_delta,rbc_delta,rdw_delta,wbc_delta,inr_delta,pt_delta,ptt_delta
0,1,58.441631,92.0,170.0,11,0,0,0,0,0,1,1,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,13.0,26.0,14.0,8.7,106.0,0.8,102.0,141.0,4.1,35.5,11.3,31.7,31.9,99.0,286.0,3.57,16.6,12.0,1.8,20.14,46.3,1.2,8.2,0.0,0.6,-1.0,-0.1,-61.0,0.0,-0.3,-0.5,-0.66,3.06,-0.9,11.8,69.4,-0.57,3.02,1.58,-0.02,0.02,-5.46
1,0,79.748598,55.0,170.0,5,0,0,0,0,0,1,0,1,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,15.0,22.0,17.0,7.9,103.0,0.7,155.0,136.0,3.9,41.8,14.2,31.7,34.1,93.0,248.0,4.49,15.1,27.3,1.1,11.60,27.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00
2,0,81.717397,59.0,170.0,8,0,0,1,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,16.0,19.0,60.0,7.7,113.0,2.3,97.0,143.0,5.1,30.9,10.1,30.3,32.5,93.0,153.0,3.32,13.3,9.3,1.5,15.70,27.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00
4,0,84.191979,79.1,160.0,6,1,0,1,1,0,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,17.0,12.0,44.0,10.1,112.0,2.2,96.0,136.0,4.7,27.9,8.4,26.9,30.1,89.0,157.0,3.12,17.6,8.8,1.2,13.40,32.6,2.0,-3.0,-2.0,-0.8,5.0,-0.3,-57.0,4.0,-0.1,-6.0,-1.70,-0.30,0.3,-2.0,-38.0,-0.59,0.20,-4.10,0.10,1.40,4.50
6,0,58.039790,61.5,150.0,5,0,0,1,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,15.0,23.0,27.0,8.4,90.0,1.3,143.0,128.0,3.4,21.1,6.9,30.0,32.7,92.0,48.0,2.30,20.9,7.6,1.7,18.60,84.0,-2.0,2.0,-1.0,0.2,0.0,-0.1,11.0,0.0,-0.2,-0.6,-0.10,0.30,0.4,0.0,26.0,-0.06,-0.20,0.40,-0.30,-2.70,-66.00


In [30]:
# Perform normalization using data from X_train to transform other X_test and X_holdout

from sklearn.preprocessing import StandardScaler

num_cols = X_train.columns[X_train.dtypes.apply(lambda c: np.issubdtype(c, np.number))]
# print(num_cols)
scaler = StandardScaler()
X_train[num_cols] = scaler.fit_transform(X_train[num_cols])
X_test[num_cols] = scaler.transform(X_test[num_cols])
X_holdout[num_cols] = scaler.transform(X_holdout[num_cols])


In [31]:
X_train.head()

,gender,admission_age,weight_admit,height,charlson_score,atrial_fibrillation,malignant_cancer,chf,ckd,cld,copd,diabetes,hypertension,ihd,stroke,race_encode_African,race_encode_Asian,race_encode_Caucasian,race_encode_Hispanic,race_encode_Not Specified,race_encode_South American,admission_type_DIRECT EMER.,admission_type_DIRECT OBSERVATION,admission_type_ELECTIVE,admission_type_EU OBSERVATION,admission_type_EW EMER.,admission_type_OBSERVATION ADMIT,admission_type_SURGICAL SAME DAY ADMISSION,admission_type_URGENT,first_careunit_Cardiac Vascular Intensive Care Unit (CVICU),first_careunit_Coronary Care Unit (CCU),first_careunit_Medical Intensive Care Unit (MICU),first_careunit_Medical/Surgical Intensive Care Unit (MICU/SICU),first_careunit_Neuro Intermediate,first_careunit_Neuro Stepdown,first_careunit_Neuro Surgical Intensive Care Unit (Neuro SICU),first_careunit_Surgical Intensive Care Unit (SICU),first_careunit_Trauma SICU (TSICU),aniongap_last,bicarbonate_last,bun_last,calcium_last,chloride_last,creatinine_last,glucose_last,sodium_last,potassium_last,hematocrit_last,hemoglobin_last,mch_last,mchc_last,mcv_last,platelet_last,rbc_last,rdw_last,wbc_last,inr_last,pt_last,ptt_last,aniongap_delta,bicarbonate_delta,bun_delta,calcium_delta,chloride_delta,creatinine_delta,glucose_delta,sodium_delta,potassium_delta,hematocrit_delta,hemoglobin_delta,mch_delta,mchc_delta,mcv_delta,platelet_delta,rbc_delta,rdw_delta,wbc_delta,inr_delta,pt_delta,ptt_delta
0,1,-0.563436,0.232553,0.031491,1.774812,-0.604490,-0.385171,-0.724663,-0.861473,-0.356470,1.588920,1.412212,0.623726,-0.944235,-0.374677,-0.388041,-0.164393,-1.400080,-0.197225,2.453196,-0.026464,-0.211895,-0.011832,6.041849,-0.027756,-1.127697,-0.369925,-0.273432,-0.468075,-0.363662,-0.37078,-0.600822,-0.489721,-0.083527,-0.054295,-0.148916,-0.421541,2.692582,-0.442190,0.576355,-0.798280,0.498018,0.399128,-0.589487,-0.636342,0.559352,-0.234946,0.906842,0.709281,0.745233,-0.609654,1.201147,0.941633,0.294369,0.331701,-0.014377,0.230621,0.320217,0.028382,0.494428,2.115475,-0.042766,0.858249,-0.409045,-0.115540,-0.528519,-0.012926,-0.296677,0.280702,-0.030933,1.628666,-0.811262,2.368328,1.062702,-0.610388,1.725311,0.190473,-0.025679,-0.001862,-0.323898
1,0,0.802145,-1.059453,0.031491,-0.277074,-0.604490,-0.385171,-0.724663,-0.861473,-0.356470,1.588920,-0.708109,0.623726,1.059058,-0.374677,-0.388041,-0.164393,0.714245,-0.197225,-0.407631,-0.026464,-0.211895,-0.011832,-0.165512,-0.027756,0.886763,-0.369925,-0.273432,-0.468075,-0.363662,-0.37078,-0.600822,-0.489721,-0.083527,-0.054295,-0.148916,-0.421541,2.692582,0.026102,-0.229090,-0.673474,-0.529005,-0.056380,-0.641715,0.244234,-0.418894,-0.549560,2.093332,2.322414,0.745233,0.811007,0.298344,0.563854,1.736976,-0.346122,1.773410,-0.745804,-0.837501,-0.978437,0.199433,0.037460,-0.042766,0.135650,-0.199086,-0.011309,0.139918,-0.012926,0.082719,0.377469,0.352737,-0.002805,-0.160136,0.108856,0.235954,0.336269,-0.110272,-0.011434,-0.002367,-0.004129,-0.103047
2,0,0.928327,-0.919777,0.031491,0.748869,-0.604490,-0.385171,1.379952,1.160802,-0.356470,-0.629358,-0.708109,-1.603267,1.059058,-0.374677,-0.388041,-0.164393,0.714245,-0.197225,-0.407631,-0.026464,-0.211895,-0.011832,-0.165512,-0.027756,0.886763,-0.369925,-0.273432,-0.468075,-0.363662,-0.37078,-0.600822,2.041980,-0.083527,-0.054295,-0.148916,-0.421541,-0.371391,0.260248,-0.833174,1.115405,-0.785760,1.461979,0.193928,-0.719415,0.950650,1.338128,0.040515,0.041777,0.177507,-0.222201,0.298344,-0.380594,-0.097644,-1.159509,-0.329869,-0.187847,-0.281688,-0.968004,0.199433,0.037460,-0.042766,0.135650,-0.199086,-0.011309,0.139918,-0.012926,0.082719,0.377469,0.352737,-0.002805,-0.160136,0.108856,0.235954,0.336269,-0.110272,-0.011434,-0.002367,-0.004129,-0.103047
4,0,1.086925,-0.217903,-1.230875,0.064907,1.654286,-0.385171,1.379952,1.160802,-0.356470,-0.629358,1.412212,0.623726,-0.944235,-0.374677,-0.388041,-0.164393,0.714245,-0.197225,-0.407631,-0.026464,-0.211895,-0.011832,-0.165512,-0.027756,-1.127697,

In [32]:
X_test.describe()

,admission_age,weight_admit,height,charlson_score,atrial_fibrillation,malignant_cancer,chf,ckd,cld,copd,diabetes,hypertension,ihd,stroke,race_encode_African,race_encode_Asian,race_encode_Caucasian,race_encode_Hispanic,race_encode_Not Specified,race_encode_South American,admission_type_DIRECT EMER.,admission_type_DIRECT OBSERVATION,admission_type_ELECTIVE,admission_type_EU OBSERVATION,admission_type_EW EMER.,admission_type_OBSERVATION ADMIT,admission_type_SURGICAL SAME DAY ADMISSION,admission_type_URGENT,first_careunit_Cardiac Vascular Intensive Care Unit (CVICU),first_careunit_Coronary Care Unit (CCU),first_careunit_Medical Intensive Care Unit (MICU),first_careunit_Medical/Surgical Intensive Care Unit (MICU/SICU),first_careunit_Neuro Intermediate,first_careunit_Neuro Stepdown,first_careunit_Neuro Surgical Intensive Care Unit (Neuro SICU),first_careunit_Surgical Intensive Care Unit (SICU),first_careunit_Trauma SICU (TSICU),aniongap_last,bicarbonate_last,bun_last,calcium_last,chloride_last,creatinine_last,glucose_last,sodium_last,potassium_last,hematocrit_last,hemoglobin_last,mch_last,mchc_last,mcv_last,platelet_last,rbc_last,rdw_last,wbc_last,inr_last,pt_last,ptt_last,aniongap_delta,bicarbonate_delta,bun_delta,calcium_delta,chloride_delta,creatinine_delta,glucose_delta,sodium_delta,potassium_delta,hematocrit_delta,hemoglobin_delta,mch_delta,mchc_delta,mcv_delta,platelet_delta,rbc_delta,rdw_delta,wbc_delta,inr_delta,pt_delta,ptt_delta
count,4083.000000,4083.000000,4083.000000,4083.000000,4083.000000,4083.000000,4083.000000,4083.000000,4083.000000,4083.000000,4083.000000,4083.000000,4083.000000,4083.000000,4083.000000,4083.000000,4083.000000,4083.000000,4083.000000,4083.000000,4083.000000,4083.000000,4083.000000,4083.000000,4083.000000,4083.000000,4083.000000,4083.000000,4083.000000,4083.000000,4083.000000,4083.000000,4083.000000,4083.000000,4083.000000,4083.000000,4083.000000,4083.000000,4083.000000,4083.000000,4083.000000,4083.000000,4083.000000,4083.000000,4083.000000,4083.000000,4083.000000,4083.000000,4083.000000,4083.000000,4083.000000,4083.000000,4083.000000,4083.000000,4083.000000,4083.000000,4083.000000,4083.000000,4083.000000,4083.000000,4083.000000,4083.000000,4083.000000,4083.000000,4083.000000,4083.000000,4083.000000,4083.000000,4083.000000,4083.000000,4083.000000,4083.000000,4083.000000,4083.000000,4083.000000,4083.000000,4083.000000,4083.000000,4083.000000
mean,-0.010129,-0.005150,-0.017707,-0.030828,0.006259,-0.009846,0.000071,0.009745,0.002064,-0.011087,-0.001335,-0.017700,-0.009562,-0.019847,-0.005332,0.043700,0.003256,-0.010157,-0.013856,0.001320,-0.018656,0.029575,0.033646,-0.018926,-0.000823,0.028993,-0.012544,-0.019003,0.013797,0.010909,-0.014963,-0.004835,0.005053,0.013567,-0.004339,0.007482,-0.008186,-0.001492,-0.014001,-0.015211,-0.007147,-0.006766,-0.033986,0.006716,-0.018828,0.011630,0.018105,0.023495,0.007676,0.012573,0.002061,0.018760,0.022990,-0.020466,0.041510,0.027546,0.028144,0.017922,0.014835,-0.009747,0.003423,-0.004256,-0.002288,0.024635,0.023208,0.005808,0.015113,-0.009878,-0.001759,-0.019924,0.002632,-0.021557,-0.021552,0.007092,0.002167,0.021482,0.008711,0.011072,0.005431
std,0.992835,0.931019,1.002064,0.985506,1.003383,0.989127,1.000146,1.001533,1.002645,0.994726,0.999651,1.008600,0.999528,0.976886,0.994255,1.121177,0.999000,0.975003,0.985750,1.024749,0.956967,1.870632,1.094019,0.564264,1.000221,1.032991,0.978587,0.983957,1.016358,1.012675,0.992020,0.996351,1.029712,1.117700,0.985764,1.007367,0.990541,1.011186,0.995225,1.017726,0.963002,0.973588,0.914046,1.135962,0.990946,1.015970,1.018667,1.026705,1.033355,1.015202,1.033108,0.991779,1.014870,1.006686,1.210431,1.003718,1.007611,1.015759,1.004626,1.035547,1.022225,0.953165,1.001274,0.959411,0.988080,0.996989,0.976850,1.025092,1.030398,1.046105,0.987398,1.025849,1.017553,1.028550,0.975145,1.038227,0.949292,0.909762,1.000290
min,-3.110447,-2.945083,-5.396685,-1.986980,-0.604490,-0.385171,-0.724663,-0.861473,-0.356470,-0.629358,-0.708109,-1.603

In [33]:
print(len(X_train.columns))
print(len(num_cols))

80
79


In [34]:
#dropping icu_outcome column

# dynamic_train_df = dynamic_train_df.drop('icu_outcome', axis=1, inplace=True)

In [35]:
# converting pre-proceesed & splitted static and dynamic df to CSV

# static_train_df.to_csv('static_train_df.csv')
# static_test_df.to_csv('static_test_df.csv')
# static_holdout_df.to_csv('static_holdout_df.csv')
# dynamic_train_df.to_csv('dynamic_train_df.csv')
# dynamic_test_df.to_csv('dynamic_test_df.csv')
# dynamic_holdout_df.to_csv('dynamic_holdout_df.csv')

In [36]:
# static_train_df.to_excel('static_train_df.xlsx', index=False)
# static_test_df.to_excel('static_test_df.xlsx', index=False)
# static_holdout_df.to_excel('static_holdout_df.xlsx', index=False)

# dynamic_train_df.to_excel('dynamic_train_df.xlsx', index=False)
# dynamic_test_df.to_excel('dynamic_test_df.xlsx', index=False)
# dynamic_holdout_df.to_excel('dynamic_holdout_df.xlsx', index=False)

# Applying vanilla linear regression without regularization

In [46]:
from sklearn.linear_model import LinearRegression

linear = LinearRegression()
linear.fit(X_train, y_train)

LinearRegression()

In [47]:
# Testing linear regression with X_test

from sklearn.metrics import mean_squared_error

y_train_pred = linear.predict(X_train)
mse_train_linear = mean_squared_error(y_train, y_train_pred)
print("Linear Regression y_train MSE:", mse_train_linear)

y_test_pred = linear.predict(X_test)
mse_test_linear = mean_squared_error(y_test, y_test_pred)
print("Linear Regression y_test MSE:", mse_test_linear)

y_holdout_pred = linear.predict(X_holdout)
mse_holdout_linear = mean_squared_error(y_holdout, y_holdout_pred)
print("Linear Regression y_holdout MSE:", mse_holdout_linear)

Linear Regression y_train MSE: 34.77882305781748
Linear Regression y_test MSE: 34.31394154777961
Linear Regression y_holdout MSE: 28.726688442049475


In [48]:
linear_coef = linear.coef_

feature_list = X_train.columns.tolist()

linear_coefficients_df = pd.DataFrame({'Feature': feature_list, 'Coefficient': linear_coef})

print(linear_coefficients_df)

                                              Feature   Coefficient
0                                              gender  4.222679e-01
1                                       admission_age -3.497634e-01
2                                        weight_admit  1.791221e-01
3                                              height -1.740059e-01
4                                      charlson_score  5.167330e-02
5                                 atrial_fibrillation  2.519023e-01
6                                    malignant_cancer -1.658420e-01
7                                                 chf  3.277963e-02
8                                                 ckd -6.214123e-02
9                                                 cld -1.420075e-01
10                                               copd  1.648353e-01
11                                           diabetes -8.480892e-02
12                                       hypertension -1.290660e-01
13                                              

# Applying linear regression with L1 Lasso regularization

In [49]:
from sklearn.linear_model import Lasso, Ridge

# Training with L1 lasso regularization

lasso_model = Lasso(alpha=0.1)
lasso_model.fit(X_train, y_train)

coefficients = lasso_model.coef_

print("Coefficients:", coefficients)

Coefficients: [ 0.         -0.1824318   0.11797065 -0.         -0.          0.1404416
 -0.07038049  0.         -0.         -0.          0.05848097 -0.
 -0.06193174 -0.12571011  0.08788111  0.          0.         -0.
 -0.          0.0960847  -0.         -0.         -0.         -0.01595258
 -0.          0.         -0.         -0.          0.43798101  0.
 -0.         -0.         -0.01606012  0.         -0.          0.28229859
  0.15163604  0.16630174  0.11139223 -0.08496248  0.17681297 -0.2033863
  0.         -0.          0.          0.06310156  0.          0.01508164
  0.01337972 -0.          0.         -0.         -0.          0.
  0.          0.09729424  0.          0.          0.09057632  0.
 -0.          0.          0.06307047  0.          0.14385627 -0.
  0.         -0.         -0.04283672 -0.         -0.          0.06396835
 -0.04760033 -0.         -0.         -0.         -0.          0.
  0.         -0.        ]


In [50]:
feature_list = X_train.columns.tolist()

coefficients_df = pd.DataFrame({'Feature': feature_list, 'Coefficient': coefficients})

print(coefficients_df)

                                              Feature  Coefficient
0                                              gender     0.000000
1                                       admission_age    -0.182432
2                                        weight_admit     0.117971
3                                              height    -0.000000
4                                      charlson_score    -0.000000
5                                 atrial_fibrillation     0.140442
6                                    malignant_cancer    -0.070380
7                                                 chf     0.000000
8                                                 ckd    -0.000000
9                                                 cld    -0.000000
10                                               copd     0.058481
11                                           diabetes    -0.000000
12                                       hypertension    -0.061932
13                                                ihd    -0.12

In [51]:
# Calculating MSE for L1 Lasso Regression

y_train_pred = lasso_model.predict(X_train)
mse_train_lasso = mean_squared_error(y_train, y_train_pred)
print("L1 Lasso Regression y_train MSE:", mse_train_lasso)

y_test_pred = lasso_model.predict(X_test)
mse_test_lasso = mean_squared_error(y_test, y_test_pred)
print("L1 Lasso Regression y_test MSE:", mse_test_lasso)

y_holdout_pred = lasso_model.predict(X_holdout)
mse_holdout_lasso = mean_squared_error(y_holdout, y_holdout_pred)
print("L1 Lasso Regression y_holdout MSE:", mse_holdout_lasso)

L1 Lasso Regression y_train MSE: 35.25072128289566
L1 Lasso Regression y_test MSE: 34.51365629089378
L1 Lasso Regression y_holdout MSE: 29.10456367260799


# Applying linear regression with L2 Ridge regularization

In [52]:
# Training with L2 ridge regularization

ridge_model = Ridge(alpha=0.1)
ridge_model.fit(X_train, y_train)


ridge_coef = ridge_model.coef_

print("Coefficients:", ridge_coef)

Coefficients: [ 4.15569140e-01 -3.49377014e-01  1.79173107e-01 -1.73324552e-01
  5.02271930e-02  2.51780605e-01 -1.65984576e-01  3.41461717e-02
 -6.28742871e-02 -1.42254109e-01  1.64686850e-01 -8.29352126e-02
 -1.30630115e-01 -2.15799555e-01  1.82762505e-01  3.12039836e-02
  2.91069667e-02 -6.67280416e-02 -6.28721035e-02  8.50448362e-02
 -5.34898465e-02 -5.41678728e-02 -4.17737800e-02 -1.50390239e-01
 -9.51871152e-02 -8.58886169e-02 -1.71320807e-01 -7.93694748e-02
  4.08701865e-01  2.94692897e-03 -1.76842309e-01 -1.40504247e-01
 -1.72454507e-01  4.86491205e-02 -4.51609663e-02  3.53647244e-01
  1.93770488e-01  1.97239701e-01  1.57778021e-01 -1.10510957e-01
  3.97842584e-01 -4.15854760e-01 -1.11070252e-01 -2.37272316e-01
  8.14342557e-02  2.19802478e-01  5.18546258e-02  2.87372931e+00
 -1.76348588e+00 -1.15733331e+00  1.17146038e+00  6.03508487e-01
  1.25409300e-02 -1.06432491e+00  1.70929675e-01  1.52052118e-01
  1.08817588e-01 -9.28047286e-02  2.34186458e-01 -1.24470550e-02
 -6.6885272

In [53]:
feature_list = X_train.columns.tolist()

ridge_coefficients_df = pd.DataFrame({'Feature': feature_list, 'Coefficient': ridge_coef})

print(ridge_coefficients_df)

                                              Feature  Coefficient
0                                              gender     0.415569
1                                       admission_age    -0.349377
2                                        weight_admit     0.179173
3                                              height    -0.173325
4                                      charlson_score     0.050227
5                                 atrial_fibrillation     0.251781
6                                    malignant_cancer    -0.165985
7                                                 chf     0.034146
8                                                 ckd    -0.062874
9                                                 cld    -0.142254
10                                               copd     0.164687
11                                           diabetes    -0.082935
12                                       hypertension    -0.130630
13                                                ihd    -0.21

In [54]:
# Calculating MSE for L1 Lasso Regression

y_train_pred = ridge_model.predict(X_train)
mse_train_ridge = mean_squared_error(y_train, y_train_pred)
print("L2 Ridge Regression y_train MSE:", mse_train_ridge)

y_test_pred = ridge_model.predict(X_test)
mse_test_ridge = mean_squared_error(y_test, y_test_pred)
print("L2 Ridge Regression y_test MSE:", mse_test_ridge)

y_holdout_pred = ridge_model.predict(X_holdout)
mse_holdout_ridge = mean_squared_error(y_holdout, y_holdout_pred)
print("L2 Ridge Regression y_holdout MSE:", mse_holdout_ridge)

L2 Ridge Regression y_train MSE: 34.77879606503892
L2 Ridge Regression y_test MSE: 34.31337837312319
L2 Ridge Regression y_holdout MSE: 28.726003009235885


# Applying linear regression using pytorch

In [43]:
import torch
import torch.nn as nn
from torch.optim import SGD #gradient descent optimizer

# NumPy for math operations, and Pandas for processing tabular data.
import numpy as np
import pandas as pd

# Plotly plotting package
import plotly.graph_objects as go
import plotly.express as px

In [44]:
X_train['gender'] = X_train['gender'].astype(int)
X_test['gender'] = X_test['gender'].astype(int)
X_holdout['gender'] = X_holdout['gender'].astype(int)
print(X_train.dtypes)

gender                                                               int32
admission_age                                                      float64
weight_admit                                                       float64
height                                                             float64
charlson_score                                                     float64
atrial_fibrillation                                                float64
malignant_cancer                                                   float64
chf                                                                float64
ckd                                                                float64
cld                                                                float64
copd                                                               float64
diabetes                                                           float64
hypertension                                                       float64
ihd                      

In [45]:
X_train_tensor = torch.tensor(X_train.to_numpy(),dtype=torch.float32)
m,n = X_train_tensor.shape
y_train_tensor = torch.tensor(y_train.to_numpy(),dtype=torch.float32).reshape(m,1)

X_test_tensor = torch.tensor(X_test.to_numpy(),dtype=torch.float32)
m,n = X_test_tensor.shape
y_test_tensor = torch.tensor(y_test.to_numpy(),dtype=torch.float32).reshape(m,1)

X_holdout_tensor = torch.tensor(X_holdout.to_numpy(),dtype=torch.float32)
m,n = X_holdout_tensor.shape
y_holdout_tensor = torch.tensor(y_holdout.to_numpy(),dtype=torch.float32).reshape(m,1)

In [46]:
print('Training input has size: ',X_train_tensor.shape)
print('y_Training input has size: ',y_train_tensor.shape)

Training input has size:  torch.Size([14289, 80])
y_Training input has size:  torch.Size([14289, 1])


In [59]:
h = torch.nn.Linear(
    in_features=80,
    out_features=1,
    bias=True
)

# For torch SGD, we need to tell it which parameter we what to optimize.
GD_optimizer = torch.optim.SGD(lr=0.05, params=h.parameters())
J_MSE = torch.nn.MSELoss()

# Apply gradient descent 10000 times
nIter = 10000
printInterval = 500

# PyTorch optimization steps:
# 1. Clear gradient
# 2. Calculate model and loss values (so-called forward path)
# 3. Calculate gradient of loss (so-called backward path)
# 4. Ask optimizer to update parameters of model
for i in range(nIter):
    # Step 1
    GD_optimizer.zero_grad()
    # Step 2
    pred = h(X_train_tensor)
    loss = J_MSE(pred,y_train_tensor)
    # Step 3
    loss.backward()
    # Step 4
    GD_optimizer.step()
    # Print loss value to track optimization progress
    if i == 0 or ((i+1) % printInterval) == 0:
        # We take square root of MSE (PyTorch internally averaged in J_MSE)
        # so that scale of printout is same as scale of y values.
        print('Iter {} : average rooted training MSE {:.3f}'.format(i+1,torch.sqrt(loss).item()))

Iter 1 : average rooted training MSE 7.838
Iter 500 : average rooted training MSE 5.899
Iter 1000 : average rooted training MSE 5.899
Iter 1500 : average rooted training MSE 5.899
Iter 2000 : average rooted training MSE 5.899
Iter 2500 : average rooted training MSE 5.898
Iter 3000 : average rooted training MSE 5.898
Iter 3500 : average rooted training MSE 5.898
Iter 4000 : average rooted training MSE 5.898
Iter 4500 : average rooted training MSE 5.898
Iter 5000 : average rooted training MSE 5.898
Iter 5500 : average rooted training MSE 5.898
Iter 6000 : average rooted training MSE 5.898
Iter 6500 : average rooted training MSE 5.898
Iter 7000 : average rooted training MSE 5.898
Iter 7500 : average rooted training MSE 5.898
Iter 8000 : average rooted training MSE 5.898
Iter 8500 : average rooted training MSE 5.898
Iter 9000 : average rooted training MSE 5.898
Iter 9500 : average rooted training MSE 5.898
Iter 10000 : average rooted training MSE 5.898


In [60]:
# Calculate testing results.

with torch.no_grad():
    y_train_pred = h(X_train_tensor)
with torch.no_grad():
    y_test_pred = h(X_test_tensor)
with torch.no_grad():
    y_holdout_pred = h(X_holdout_tensor)


mse_train_torch = mean_squared_error(y_train, y_train_pred)
print("pyTorch y_train MSE:", mse_train_torch)

mse_test_torch = mean_squared_error(y_test, y_test_pred)
print("pyTorch y_test MSE:", mse_test_torch)

mse_holdout_torch = mean_squared_error(y_holdout, y_holdout_pred)
print("pyTorch y_holdout MSE:", mse_holdout_torch)

pyTorch y_train MSE: 34.78137918280998
pyTorch y_test MSE: 34.318655703878974
pyTorch y_holdout MSE: 28.729938428101097


# Random Forest Regressor

In [61]:
from sklearn.ensemble import RandomForestRegressor

rf_regressor = RandomForestRegressor(n_estimators=100, max_depth=8, min_samples_split=10, random_state=42)

rf_regressor.fit(X_train, y_train)

RandomForestRegressor(max_depth=8, min_samples_split=10, random_state=42)

In [62]:
feature_importances = rf_regressor.feature_importances_
feature_importance_df = pd.DataFrame({'Feature': X_train.columns, 'Importance': feature_importances})
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)
print(feature_importance_df)

                                              Feature  Importance
3                                              height    0.256148
41                                       calcium_last    0.069116
2                                        weight_admit    0.056441
62                                      calcium_delta    0.052384
49                                           mch_last    0.036870
48                                    hemoglobin_last    0.032894
43                                    creatinine_last    0.027969
55                                           wbc_last    0.026269
40                                           bun_last    0.022623
79                                          ptt_delta    0.021700
58                                           ptt_last    0.021203
76                                          wbc_delta    0.020118
1                                       admission_age    0.017098
46                                     potassium_last    0.016636
66        

In [63]:
# Calculating MSE for Random Forest Regressor

y_train_pred = rf_regressor.predict(X_train)
mse_train_rfregressor = mean_squared_error(y_train, y_train_pred)
print("RF regressor y_train MSE:", mse_train_rfregressor)

y_test_pred = rf_regressor.predict(X_test)
mse_test_rfregressor = mean_squared_error(y_test, y_test_pred)
print("RF regressor y_test MSE:", mse_test_ridge)

y_holdout_pred = rf_regressor.predict(X_holdout)
mse_holdout_rfregressor = mean_squared_error(y_holdout, y_holdout_pred)
print("RF regressor y_holdout MSE:", mse_holdout_rfregressor)

RF regressor y_train MSE: 27.115352504934187
RF regressor y_test MSE: 34.31337837312319
RF regressor y_holdout MSE: 28.1080519973965


In [64]:
# Experimenting with removal of height and weight from the X datasets and training RF regressor again

# X_train = X_train.drop(columns=['height', 'weight_admit'])
# X_test = X_test.drop(columns=['height', 'weight_admit'])
# X_holdout = X_holdout.drop(columns=['height', 'weight_admit'])

In [65]:
# rf_regressor = RandomForestRegressor(n_estimators=100, max_depth=8, min_samples_split=10)

# rf_regressor.fit(X_train, y_train)

In [66]:
feature_importances = rf_regressor.feature_importances_
feature_importance_df = pd.DataFrame({'Feature': X_train.columns, 'Importance': feature_importances})
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)
print(feature_importance_df)

                                              Feature  Importance
3                                              height    0.256148
41                                       calcium_last    0.069116
2                                        weight_admit    0.056441
62                                      calcium_delta    0.052384
49                                           mch_last    0.036870
48                                    hemoglobin_last    0.032894
43                                    creatinine_last    0.027969
55                                           wbc_last    0.026269
40                                           bun_last    0.022623
79                                          ptt_delta    0.021700
58                                           ptt_last    0.021203
76                                          wbc_delta    0.020118
1                                       admission_age    0.017098
46                                     potassium_last    0.016636
66        

In [67]:
# Calculating MSE for Random Forest Regressor

y_train_pred = rf_regressor.predict(X_train)
mse_train_rfregressor = mean_squared_error(y_train, y_train_pred)
print("RF regressor y_train MSE:", mse_train_rfregressor)

y_test_pred = rf_regressor.predict(X_test)
mse_test_rfregressor = mean_squared_error(y_test, y_test_pred)
print("RF regressor y_test MSE:", mse_test_ridge)

y_holdout_pred = rf_regressor.predict(X_holdout)
mse_holdout_rfregressor = mean_squared_error(y_holdout, y_holdout_pred)
print("RF regressor y_holdout MSE:", mse_holdout_rfregressor)

RF regressor y_train MSE: 27.115352504934187
RF regressor y_test MSE: 34.31337837312319
RF regressor y_holdout MSE: 28.1080519973965


# Support Vector Regressor 

In [68]:
from sklearn.svm import SVR

svr_regressor = SVR()

svr_regressor.fit(X_train, y_train)

SVR()

In [69]:
# Calculating MSE for Support Vector Regressor

y_train_pred = svr_regressor.predict(X_train)
mse_train_SVR = mean_squared_error(y_train, y_train_pred)
print("Support Vector regressor y_train MSE:", mse_train_SVR)

y_test_pred = svr_regressor.predict(X_test)
mse_test_SVR = mean_squared_error(y_test, y_test_pred)
print("Support Vector regressor y_test MSE:", mse_test_SVR)

y_holdout_pred = svr_regressor.predict(X_holdout)
mse_holdout_SVR = mean_squared_error(y_holdout, y_holdout_pred)
print("Support Vector regressor y_holdout MSE:", mse_holdout_SVR)

Support Vector regressor y_train MSE: 35.65656079625781
Support Vector regressor y_test MSE: 36.89767996991137
Support Vector regressor y_holdout MSE: 30.601147816237145


# Deep Neural Network Regression

In [38]:
import torch
import torch.nn as nn
import torch.optim as optim

In [39]:
X_train['gender'] = X_train['gender'].astype(int)
X_test['gender'] = X_test['gender'].astype(int)
X_holdout['gender'] = X_holdout['gender'].astype(int)
print(X_train.dtypes)

gender                                                               int32
admission_age                                                      float64
weight_admit                                                       float64
height                                                             float64
charlson_score                                                     float64
atrial_fibrillation                                                float64
malignant_cancer                                                   float64
chf                                                                float64
ckd                                                                float64
cld                                                                float64
copd                                                               float64
diabetes                                                           float64
hypertension                                                       float64
ihd                      

In [40]:
# X = torch.tensor(X_train.to_numpy(),dtype=torch.float32)
# m,n = X.shape
# y = torch.tensor(y_train.to_numpy(),dtype=torch.float32).reshape(m,1)

X_train_tensor = torch.tensor(X_train.to_numpy(),dtype=torch.float32)
m,n = X_train_tensor.shape
y_train_tensor = torch.tensor(y_train.to_numpy(),dtype=torch.float32).reshape(m,1)

X_test_tensor = torch.tensor(X_test.to_numpy(),dtype=torch.float32)
m,n = X_test_tensor.shape
y_test_tensor = torch.tensor(y_test.to_numpy(),dtype=torch.float32).reshape(m,1)

X_holdout_tensor = torch.tensor(X_holdout.to_numpy(),dtype=torch.float32)
m,n = X_holdout_tensor.shape
y_holdout_tensor = torch.tensor(y_holdout.to_numpy(),dtype=torch.float32).reshape(m,1)

In [64]:
class NN_Regressor(nn.Module):
    def __init__(self, input_size, output_size): # set the arguments you'd need, including activation function
        super(NN_Regressor, self).__init__()
        self.input_size = input_size
        self.output_size = output_size

        self.linear = nn.Linear(self.input_size, 32)
        self.linear2 = nn.Linear(32, 8)
        self.linear3 = nn.Linear(8, output_size)

        self.relu = nn.ReLU()

    def forward(self, x):
        linear = self.linear(x)
        linear2 = self.linear2(self.relu(linear))
        linear3 = self.linear3(self.relu(linear2))
        #linear3 = self.linear3(linear2)
        return self.relu(linear3)

In [67]:
# 2 hidden layers NN

class NN_Regressor(nn.Module):
    def __init__(self, input_size, output_size): # set the arguments you'd need, including activation function
        super(NN_Regressor, self).__init__()
        self.input_size = input_size
        self.output_size = output_size

        self.linear = nn.Linear(self.input_size, 64)
        self.linear2 = nn.Linear(64, 32)
        self.linear3 = nn.Linear(32, 32)
        self.linear4 = nn.Linear(32, output_size)

        self.relu = nn.ReLU()

    def forward(self, x):
        linear = self.linear(x)
        linear2 = self.linear2(self.relu(linear))
        linear3 = self.linear3(self.relu(linear2))
        linear4 = self.linear4(self.relu(linear3))
        return self.relu(linear4)

def train_model(x_train, y_train, epochs=200):
    model = NN_Regressor(80, 1)
    optimiser = optim.Adam(model.parameters(), lr=0.001)
    loss_fn = nn.MSELoss()

    for i in range(epochs):
        # reset gradients to 0
        optimiser.zero_grad()

        # get predictions
        y_pred = model.forward(x_train)
        
        # compute loss (uncomment the next line and fill right hand side)
        abs_loss = loss_fn(y_pred, y_train) 

        # backpropagate
        abs_loss.backward()
    
        # update the model weights
        optimiser.step()
        
        print (f"{i:5d}", abs_loss.item(), sep='\t')
        
    return model
                
NNRegressor = train_model(X_train_tensor, y_train_tensor)

    0	60.14265060424805
    1	60.079959869384766
    2	59.978519439697266
    3	59.842193603515625
    4	59.680908203125
    5	59.504512786865234
    6	59.31904602050781
    7	59.12621307373047
    8	58.92580795288086
    9	58.71662521362305
   10	58.497642517089844
   11	58.26771926879883
   12	58.02590560913086
   13	57.77120590209961
   14	57.502620697021484
   15	57.2191047668457
   16	56.91944885253906
   17	56.602500915527344
   18	56.26703643798828
   19	55.91189956665039
   20	55.535552978515625
   21	55.136878967285156
   22	54.71456527709961
   23	54.26727294921875
   24	53.79353713989258
   25	53.29198455810547
   26	52.76145553588867
   27	52.200843811035156
   28	51.609275817871094
   29	50.9862060546875
   30	50.33122634887695
   31	49.64463806152344
   32	48.92709732055664
   33	48.17972183227539
   34	47.40468215942383
   35	46.604679107666016
   36	45.78327560424805
   37	44.945640563964844
   38	44.0976448059082
   39	43.2464599609375
   40	42.400733947753906
   41	41

In [70]:
# 2 hidden layer NN with dropout

class NN_Regressor(nn.Module):
    def __init__(self, input_size, output_size, dropout_prob): # set the arguments you'd need, including activation function
        super(NN_Regressor, self).__init__()
        self.input_size = input_size
        self.output_size = output_size

        self.layer1 = nn.Linear(self.input_size, 2056)
        self.hidden2 = nn.Linear(2056, 512)
        self.hidden3 = nn.Linear(512, 128)
        self.hidden4 = nn.Linear(128, 32)
        self.output5 = nn.Linear(32, output_size)

        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout_prob)


    def forward(self, x):
        x = self.layer1(x)
        #x = self.relu(x)
        x = self.dropout(x)
        
        x = self.hidden2(x)
        x = self.relu(x)
        x = self.dropout(x)

        x = self.hidden3(x)
        #x = self.relu(x)
        #x = self.dropout(x)

        x = self.hidden4(x)
        x = self.relu(x)
        #x = self.dropout(x)

        x = self.output5(x)
        # x = self.relu(x)
        # x = self.dropout(x)

        # x = self.hidden6(x)
        # x = self.relu(x)
        # x = self.dropout(x)

        # x = self.output7(x)
        # x = self.relu(x)

        return x

def train_model(x_train, y_train, epochs=400):
    model = NN_Regressor(80, 1, 0)
    optimiser = optim.SGD(model.parameters(), lr=0.001)
    loss_fn = nn.MSELoss() 

    for i in range(epochs):
        # reset gradients to 0
        optimiser.zero_grad()

        # get predictions
        y_pred = model.forward(x_train)
        
        # compute loss (uncomment the next line and fill right hand side)
        abs_loss = loss_fn(y_pred, y_train) 

        # backpropagate
        abs_loss.backward()
    
        # update the model weights
        optimiser.step()
        
        print (f"{i:5d}", abs_loss.item(), sep='\t')
        
    return model
                
NNRegressor = train_model(X_train_tensor, y_train_tensor)

    0	60.307010650634766
    1	60.15135955810547
    2	59.99739456176758
    3	59.844810485839844
    4	59.693355560302734
    5	59.54275894165039
    6	59.3927116394043
    7	59.24298095703125
    8	59.09321594238281
    9	58.9431266784668
   10	58.792510986328125
   11	58.64094924926758
   12	58.488346099853516
   13	58.334415435791016
   14	58.17903518676758
   15	58.02201843261719
   16	57.86335754394531
   17	57.7029914855957
   18	57.54087448120117
   19	57.37693405151367
   20	57.21113586425781
   21	57.04335403442383
   22	56.87366485595703
   23	56.70193099975586
   24	56.52806854248047
   25	56.35198974609375
   26	56.1734504699707
   27	55.99223709106445
   28	55.80812072753906
   29	55.620914459228516
   30	55.43033981323242
   31	55.236183166503906
   32	55.03814697265625
   33	54.835941314697266
   34	54.62931442260742
   35	54.41794967651367
   36	54.201541900634766
   37	53.97975158691406
   38	53.752288818359375
   39	53.51881790161133
   40	53.27903366088867
   41	53.

In [56]:
# 2 hidden layer NN with dropout - attempting to write a cleaner version

class NN_Regressor(nn.Module):
    def __init__(self, input_size, output_size, dropout_prob): # set the arguments you'd need, including activation function
        super(NN_Regressor, self).__init__()
        self.input_size = input_size
        self.output_size = output_size

        self.layer1 = nn.Linear(self.input_size, 64)
        self.hidden2 = nn.Linear(64, 32)
        self.hidden3 = nn.Linear(32, 32)
        self.hidden4 = nn.Linear(32, 16)
        self.hidden5 = nn.Linear(16, 16)
        self.hidden6 = nn.Linear(16, 8)
        self.output7 = nn.Linear(8, output_size)

        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout_prob)

    def forward(self, x):
        x = self.layer1(x)
        x = self.relu(x)
        x = self.dropout(x)
        
        x = self.hidden2(x)
        x = self.relu(x)
        x = self.dropout(x)

        x = self.hidden3(x)
        x = self.relu(x)
        x = self.dropout(x)

        x = self.hidden4(x)
        x = self.relu(x)
        x = self.dropout(x)

        x = self.hidden5(x)
        x = self.relu(x)
        x = self.dropout(x)

        x = self.hidden6(x)
        x = self.relu(x)
        x = self.dropout(x)

        x = self.output7(x)
        x = self.relu(x)

        return x


def train_model(x_train, y_train, epochs=1000):
    model = NN_Regressor(80, 1, 0.3)
    optimiser = optim.SGD(model.parameters(), lr=0.001)
    loss_fn = nn.MSELoss() 

    for i in range(epochs):
        # reset gradients to 0
        optimiser.zero_grad()

        # get predictions
        y_pred = model.forward(x_train)
        
        # compute loss (uncomment the next line and fill right hand side)
        abs_loss = loss_fn(y_pred, y_train) 

        # backpropagate
        abs_loss.backward()
    
        # update the model weights
        optimiser.step()
        
        print (f"{i:5d}", abs_loss.item(), sep='\t')
        
    return model
                
NNRegressor = train_model(X_train_tensor, y_train_tensor)

    0	60.17586135864258
    1	60.17586135864258
    2	60.17586135864258
    3	60.17586135864258
    4	60.17586135864258
    5	60.17586135864258
    6	60.17586135864258
    7	60.17586135864258
    8	60.17586135864258
    9	60.17586135864258
   10	60.17586135864258
   11	60.17586135864258
   12	60.17586135864258
   13	60.17586135864258
   14	60.17586135864258
   15	60.17586135864258
   16	60.17586135864258
   17	60.17586135864258
   18	60.17586135864258
   19	60.17586135864258
   20	60.17586135864258
   21	60.17586135864258
   22	60.17586135864258
   23	60.17586135864258
   24	60.17586135864258
   25	60.17586135864258
   26	60.17586135864258
   27	60.17586135864258
   28	60.17586135864258
   29	60.17586135864258
   30	60.17586135864258
   31	60.17586135864258
   32	60.17586135864258
   33	60.17586135864258
   34	60.17586135864258
   35	60.17586135864258
   36	60.17586135864258
   37	60.17586135864258
   38	60.17586135864258
   39	60.17586135864258
   40	60.17586135864258
   41	60.1758613

In [114]:
# def train_model(x_train, y_train, epochs=500):
#     model = NN_Regressor(80, 1, 0.5)
#     optimiser = optim.Adam(model.parameters(), lr=0.001)
#     loss_fn = nn.MSELoss()

#     for i in range(epochs):
#         # reset gradients to 0
#         optimiser.zero_grad()

#         # get predictions
#         y_pred = model.forward(x_train)
        
#         # compute loss (uncomment the next line and fill right hand side)
#         abs_loss = loss_fn(y_pred, y_train) 

#         # backpropagate
#         abs_loss.backward()
    
#         # update the model weights
#         optimiser.step()
        
#         print (f"{i:5d}", abs_loss.item(), sep='\t')
        
#     return model
                
# NNRegressor = train_model(X_train_tensor, y_train_tensor)

In [71]:
# checking MSE on testing and holdout set
from sklearn.metrics import mean_squared_error

with torch.no_grad():
    y_train_pred = NNRegressor(X_train_tensor)

mse_train_NNregressor = mean_squared_error(y_train_tensor, y_train_pred)
print("NN regressor y_train MSE:", mse_train_NNregressor)


with torch.no_grad():
    y_test_pred = NNRegressor(X_test_tensor)

mse_test_NNregressor = mean_squared_error(y_test_tensor, y_test_pred)
print("NN regressor y_test MSE:", mse_test_NNregressor)

NN regressor y_train MSE: 33.93318
NN regressor y_test MSE: 34.057457


In [69]:
with torch.no_grad():
    y_holdout_pred = NNRegressor(X_holdout_tensor)

mse_holdout_NNregressor = mean_squared_error(y_holdout_tensor, y_holdout_pred)
print("NN regressor y_test MSE:", mse_holdout_NNregressor)

NN regressor y_test MSE: 29.325027
